In [ ]:
!pip install flask pyngrok diffusers transformers accelerate safetensors huggingface_hub "imageio[ffmpeg]" --quiet

import os, uuid, warnings, logging
import torch
from flask import Flask, render_template_string, request
from pyngrok import ngrok
from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
from diffusers.utils import export_to_video
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file


warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("werkzeug").setLevel(logging.ERROR)
logging.getLogger("diffusers").setLevel(logging.ERROR)


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32
print(f"Device set to: {DEVICE}  |  DType: {DTYPE}")

STEP = 8
REPO = "ByteDance/AnimateDiff-Lightning"
CKPT = f"animatediff_lightning_{STEP}step_diffusers.safetensors"
BASE = "emilianJR/epiCRealism"

FPS = 8
CHUNKS = 4
TRIM_FRAMES = 56
HEIGHT = 512
WIDTH = 512


# HTML Template

HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
<title>Text to Video</title>
<style>
body {
    background: linear-gradient(135deg, #1f1c2c, #928dab);
    color: white; text-align: center;
    font-family: Arial, sans-serif;
}
h1, h2.glow { animation: glowAnim 1s ease-in-out infinite alternate; }
input {
    padding: 10px; width: 60%; font-size: 16px; border-radius: 5px; border: none;
}
button {
    padding: 10px 20px; font-size: 16px;
    background: linear-gradient(90deg, #00c6ff, #0072ff);
    border: none; cursor: pointer; color: white; border-radius: 5px;
}
video {
    margin-top: 20px; max-width: 512px; width: 90%;
    border: 6px solid white; border-radius: 8px;
    opacity: 0; transition: opacity 1.2s ease-in-out;
}
.download-btn {
    display: inline-block;
    margin: 6px auto 16px;
    padding: 6px 12px;
    background: linear-gradient(90deg, #00c6ff, #0072ff);
    color: white;
    border: none;
    text-decoration: none;
    font-size: 14px;
    cursor: pointer;
    border-radius: 6px;
    width: auto;
    text-align: center;
}
#loading {
    display: none; margin-top: 20px; font-size: 18px; font-weight: bold;
}
.loader-bar {
    width: 60%; height: 12px; background: #444; border-radius: 6px;
    margin: 10px auto; overflow: hidden;
}
.loader-bar-fill {
    width: 0%; height: 100%; background: linear-gradient(90deg, #00c6ff, #0072ff);
    animation: loadAnim 3s linear infinite;
}
@keyframes loadAnim {
    0% { width: 0%; } 50% { width: 100%; } 100% { width: 0%; }
}
@keyframes glowAnim {
    from { text-shadow: 0 0 5px #ff0000; }
    to   { text-shadow: 0 0 20px #ff0000; }
}
.tip-box { margin-top: 15px; font-size: 14px; color: #fffb; font-style: italic; }
</style>
<script>
let tips = [
    "🎨 Did you know? The first camera took 8 hours to capture a photo!",
    "✨ Pro Tip: Imagination has no limits!",
    "🖌️ Fun Fact: Van Gogh only sold 1 painting while alive.",
    "⚡ Creativity is intelligence having fun.",
    "🌌 Every pixel here is born from AI magic."
];
let tipIndex = 0;

function showLoading() {
    document.getElementById("loading").style.display = "block";
    changeTips();
}
function changeTips() {
    document.getElementById("tipText").innerText = tips[tipIndex];
    tipIndex = (tipIndex + 1) % tips.length;
    setTimeout(changeTips, 2000);
}
function revealVideo(el) { el.style.opacity = 1; }
</script>
</head>
<body>
<h1 class="glow">🎬 AI Text → Video Creator</h1>
<form method="POST" onsubmit="showLoading()">
    <input name="prompt" placeholder="Enter your video prompt..." required>
    <button type="submit">Generate</button>
</form>
<div id="loading">
    <div id="typingText">✨ Rendering your animation...</div>
    <div class="loader-bar"><div class="loader-bar-fill"></div></div>
    <div class="tip-box" id="tipText"></div>
</div>

{% if prompt_text %}
  <h2>Prompt: "{{ prompt_text }}"</h2>
{% endif %}

{% if video_url %}
  <h2 class="glow">🤗 Generated Video:</h2>
  <video controls onloadeddata="revealVideo(this)" src="{{ video_url }}"></video>
  <a href="{{ video_url }}" download="animation.mp4" class="download-btn">Download</a>
{% endif %}
</body>
</html>
"""


# Flask code

app = Flask(__name__)
os.makedirs("static", exist_ok=True)

print("Loading AnimateDiff pipeline...")
adapter = MotionAdapter().to(DEVICE, DTYPE)
adapter.load_state_dict(load_file(hf_hub_download(REPO, CKPT), device=DEVICE))

pipe = AnimateDiffPipeline.from_pretrained(
    BASE, motion_adapter=adapter, torch_dtype=DTYPE
).to(DEVICE)

pipe.scheduler = EulerDiscreteScheduler.from_config(
    pipe.scheduler.config, timestep_spacing="trailing", beta_schedule="linear"
)

if hasattr(pipe, "enable_vae_slicing"):
    pipe.enable_vae_slicing()
if hasattr(pipe, "enable_attention_slicing"):
    pipe.enable_attention_slicing()

def generate_video_from_prompt(prompt: str, height: int, width: int,
                               chunks: int, trim_frames: int, fps: int) -> str:
    all_frames = []
    with torch.inference_mode():
        for _ in range(chunks):
            out = pipe(
                prompt=prompt,
                guidance_scale=1.0,
                num_inference_steps=STEP,
                height=height,
                width=width,
            )
            all_frames.extend(out.frames[0])

    if trim_frames is not None and len(all_frames) > trim_frames:
        all_frames = all_frames[:trim_frames]

    filename = f"{uuid.uuid4().hex}.mp4"
    filepath = os.path.join("static", filename)
    export_to_video(all_frames, filepath, fps=fps)

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return "/" + filepath.replace("\\", "/")

@app.route("/", methods=["GET", "POST"])
def index():
    prompt_text = None
    video_url = None

    if request.method == "POST":
        prompt = request.form.get("prompt", "").strip()
        if prompt:
            prompt_text = prompt
            video_url = generate_video_from_prompt(
                prompt=prompt_text,
                height=HEIGHT,
                width=WIDTH,
                chunks=CHUNKS,
                trim_frames=TRIM_FRAMES,
                fps=FPS,
            )

    return render_template_string(
        HTML_TEMPLATE,
        prompt_text=prompt_text,
        video_url=video_url
    )


# Ngrok Tunnel

ngrok.set_auth_token(" PLEASE ADD NGROK TOKEN ")
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

app.run(port=5000, host="0.0.0.0")
